In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler

In [2]:
df = pd.read_csv('../Data/UNSW_2018_IoT_Botnet_Dataset_1.csv')

# Tampilkan info dasar
df.info()
df.head()

C:\Users\albia\AppData\Local\Temp\ipykernel_19416\588975991.py:1: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../Data/UNSW_2018_IoT_Botnet_Dataset_1.csv')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999999 entries, 0 to 999998
Data columns (total 35 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   1                  999999 non-null  int64  
 1   1526344121.188091  999999 non-null  float64
 2   e                  999999 non-null  object 
 3   arp                999999 non-null  object 
 4   192.168.100.1      999999 non-null  object 
 5   Unnamed: 5         999513 non-null  object 
 6   192.168.100.3      999999 non-null  object 
 7   Unnamed: 7         999513 non-null  object 
 8   4                  999999 non-null  int64  
 9   240                999999 non-null  int64  
 10  CON                999999 non-null  object 
 11  1526345317.184693  999999 non-null  float64
 12  9                  999999 non-null  int64  
 13  1195.996582        999999 non-null  float64
 14  0.000006           999999 non-null  float64
 15  0.000002           999999 non-null  float64
 16  Un

,1,1526344121.188091,e,arp,192.168.100.1,Unnamed: 5,192.168.100.3,Unnamed: 7,4,240,...,2,2.1,120,120.1,0.002508,0.000836,0.000836.1,0,Normal,Normal.1
0,2,1.526344e+09,e,tcp,192.168.100.7,139,192.168.100.4,36390,10,680,...,5,5,350,330,0.006190,0.002751,0.002751,0,Normal,Normal
1,3,1.526344e+09,e,udp,192.168.100.149,51838,27.124.125.250,123,2,180,...,1,1,90,90,20.590960,0.000000,0.000000,0,Normal,Normal
2,4,1.526344e+09,e,arp,192.168.100.4,NaN,192.168.100.7,NaN,10,510,...,5,5,210,300,0.006189,0.002751,0.002751,0,Normal,Normal
3,5,1.526344e+09,e,udp,192.168.100.27,58999,192.168.100.1,53,4,630,...,2,2,174,456,0.005264,0.001755,0.001755,0,Normal,Normal
4,6,1.526344e+09,e,arp,192.168.100.1,NaN,192.168.100.27,NaN,2,120,...,1,1,60,60,2724.795654,0.000000,0.000000,0,Normal,Normal


In [ ]:
df.columns = [
    "pkSeqID", "stime", "flgs", "proto", "saddr", "sport", "daddr", "dport",
    "pkts", "bytes", "state", "ltime", "seq", "dur", "mean", "stddev", "smac",
    "dmac", "sum", "min", "max", "soui", "doui", "sco", "dco", "spkts", "dpkts",
    "sbytes", "dbytes", "rate", "srate", "drate", "label", "category", "subcategory"
]

In [3]:
# Hapus spasi di awal/akhir nama kolom
df.columns = df.columns.str.strip()

In [4]:
df.drop(columns=["smac", "dmac", "soui", "doui", "sco", "dco"], inplace=True)

In [5]:
df.drop_duplicates(inplace=True)

In [6]:
df.dropna(subset=["sport", "dport"], inplace=True)

In [7]:
target = "attack"  
X = df.drop(columns=[target])
y = df[target]

In [8]:
print(y.value_counts())

attack
1    997753
0      1760
Name: count, dtype: int64


In [9]:
high_card_cat = ["saddr", "daddr", "sport", "dport"]
for col in high_card_cat:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col].astype(str))

In [10]:
low_card_cat = ["proto", "state", "flgs", "category", "subcategory"]

ohe = OneHotEncoder(sparse_output=False, handle_unknown="ignore")
ohe_result = ohe.fit_transform(X[low_card_cat])
ohe_cols = ohe.get_feature_names_out(low_card_cat)

X_ohe = pd.DataFrame(ohe_result, columns=ohe_cols, index=X.index)

In [11]:
num_cols = X.select_dtypes(include=["int64", "float64"]).columns.tolist()

# Buang high-cardinality categorical dari numerik
num_cols = [col for col in num_cols if col not in high_card_cat]

scaler = StandardScaler()
scaled_result = scaler.fit_transform(X[num_cols])

X_scaled = pd.DataFrame(scaled_result, columns=num_cols, index=X.index)

In [13]:
X_processed = pd.concat([X_scaled, X[high_card_cat], X_ohe], axis=1)

In [14]:
final_df = pd.concat([X_processed, y], axis=1)

In [15]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 999513 entries, 1 to 999999
Data columns (total 53 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   pkSeqID                   999513 non-null  float64
 1   stime                     999513 non-null  float64
 2   pkts                      999513 non-null  float64
 3   bytes                     999513 non-null  float64
 4   ltime                     999513 non-null  float64
 5   seq                       999513 non-null  float64
 6   dur                       999513 non-null  float64
 7   mean                      999513 non-null  float64
 8   stddev                    999513 non-null  float64
 9   sum                       999513 non-null  float64
 10  min                       999513 non-null  float64
 11  max                       999513 non-null  float64
 12  spkts                     999513 non-null  float64
 13  dpkts                     999513 non-null  float6

In [16]:
missing = final_df.isnull().sum()
print(missing[missing > 0])

Series([], dtype: int64)


In [17]:
final_df.head()

,pkSeqID,stime,pkts,bytes,ltime,seq,dur,mean,stddev,sum,...,flgs_e dD,flgs_e g,flgs_e r,flgs_e s,flgs_eU,category_Normal,category_Reconnaissance,subcategory_Normal,subcategory_Service_Scan,attack
1,-1.732297,-2.482046,0.004015,-0.009365,-2.474600,-1.148851,38.399038,-0.172964,-0.040363,-0.015883,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0
2,-1.732294,-2.482026,-0.012164,-0.010411,-2.482075,-1.148839,-0.041588,0.065206,-0.040468,-0.013431,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0
4,-1.732287,-2.481636,-0.008119,-0.009470,-2.478748,-1.148804,15.026018,0.310260,1.010213,-0.005917,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0
7,-1.732276,-2.481514,-0.012164,-0.010427,-2.481550,-1.148757,0.023230,-0.173102,-0.040468,-0.015890,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0
8,-1.732273,-2.481504,-0.012164,-0.010427,-2.481540,-1.148488,0.023257,-0.173102,-0.040468,-0.015890,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0


In [20]:
print(final_df['saddr'].value_counts())

saddr
0     262580
1     250513
2     230126
3     226163
5      17170
8       5088
11      3948
10      3672
7         79
4         42
6         37
19        16
12        13
15        11
13        10
17        10
14        10
18         9
16         9
9          7
Name: count, dtype: int64


In [21]:
print(final_df['daddr'].value_counts())

daddr
26     882445
29      34848
32      27141
31      24595
21       7392
        ...  
74          1
0           1
128         1
140         1
143         1
Name: count, Length: 171, dtype: int64


In [22]:
print(final_df['sport'].value_counts())

sport
32670    65542
35423    65536
35582    65536
34559    64093
37780    63481
         ...  
2311         1
3893         1
2309         1
3891         1
30815        1
Name: count, Length: 47122, dtype: int64


In [23]:
print(final_df['dport'].value_counts())

dport
17632     12675
1058       6180
17574      6059
1059       5666
44806      4478
          ...  
69962         1
18737         1
112388        1
25850         1
65545         1
Name: count, Length: 114001, dtype: int64


In [24]:
final_df.to_csv("../Data/Bot_IoT_processed.csv", index=False)

In [25]:
print("Preprocessing selesai. File disimpan sebagai Bot_IoT_processed.csv")
print("Shape sebelum:", df.shape)
print("Shape sesudah:", final_df.shape)

Preprocessing selesai. File disimpan sebagai Bot_IoT_processed.csv
Shape sebelum: (999513, 29)
Shape sesudah: (999513, 53)


In [26]:
import joblib

# simpan scaler ke file
joblib.dump(scaler, "../Data/scaler.pkl")

['../Data/scaler.pkl']